In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('data/logs.csv')
display(df.shape)
df.head()

(2065516, 16)

,id,databaseType,totalMem,timeRead,timeDelete,totalTime,timestamp,serverType,clientTotalTime,requestSize,cpuUsage,frequency,instanceType,freeMem,timeWrite,memUsage
0,--4qETfhcD,nosql,-1.0,-1.0,-1.0,-1.0,-1.0,java,-1.0,50.0,-1.0,15.0,b1,-1.0,-1.0,-1.0
1,--Lo0QBj3fC,nosql,-1.0,-1.0,-1.0,-1.0,-1.0,java,-1.0,200.0,-1.0,40.0,b1,-1.0,-1.0,-1.0
2,--P-Z58M2G,nosql,-1.0,-1.0,-1.0,-1.0,-1.0,java,-1.0,1.0,-1.0,15.0,b1,-1.0,-1.0,-1.0
3,--nUeIfVXJ,nosql,-1.0,-1.0,-1.0,-1.0,-1.0,java,-1.0,100.0,-1.0,15.0,b1,-1.0,-1.0,-1.0
4,--od3lhTv,nosql,-1.0,-1.0,-1.0,-1.0,-1.0,java,-1.0,100.0,-1.0,15.0,b1,-1.0,-1.0,-1.0


In [2]:
servers = sorted(['java', 'nodejs'])
databases = sorted(['nosql', 'sql'])
instances = sorted(['b1', 'b2', 'b4', 'b8'])

combinations = [(servers[0], databases[0]), (servers[1], databases[0]), (servers[0], databases[1]), (servers[1], databases[1])]

sizes = sorted(list(df['requestSize'].unique()))
frequencies = sorted(list(df['frequency'].unique()))
types = np.transpose([np.tile(sizes, len(frequencies)), np.repeat(frequencies, len(sizes))])

In [3]:
types = np.transpose([np.tile(sizes, len(frequencies)), np.repeat(frequencies, len(sizes))])
df.drop(['cpuUsage'], axis=1, inplace=True)
df.dropna(inplace=True)
display(df.shape)

(2065516, 15)

In [4]:
cols = df.dtypes.to_dict()

query = None

for col in cols:
    if cols[col] == 'float64':
        temp_query = (df[col] > 0)
    else:
        temp_query = (df[col] != '')
        
    temp_query = temp_query & (df[col].notnull())
    
    if query is None:
        query = temp_query
        
    else:
        query = query & temp_query
        
        
clean_df = df.loc[query]
clean_df.to_csv('data/clean.csv', index=False)
display(clean_df.shape)
clean_df.head()

(1548823, 15)

,id,databaseType,totalMem,timeRead,timeDelete,totalTime,timestamp,serverType,clientTotalTime,requestSize,frequency,instanceType,freeMem,timeWrite,memUsage
218499,VWo2CJrUe,sql,2.147484e+09,3.0,81.0,236.0,1.585877e+12,java,425.0,1.0,1.0,b1,1.887261e+09,8.0,0.121176
218500,Ge9D-u1oz,sql,2.147484e+09,3.0,91.0,299.0,1.585877e+12,java,489.0,1.0,1.0,b1,1.887121e+09,9.0,0.121241
218501,W789KeB31,sql,2.147484e+09,3.0,7.0,154.0,1.585877e+12,java,292.0,1.0,1.0,b1,1.886994e+09,10.0,0.121300
218502,YW4UJ3m4N,sql,2.147484e+09,86.0,8.0,255.0,1.585877e+12,java,448.0,1.0,1.0,b1,1.886646e+09,10.0,0.121462
218503,RRAbhpXkJ,sql,2.147484e+09,4.0,6.0,152.0,1.585877e+12,java,288.0,1.0,1.0,b1,1.886482e+09,9.0,0.121538


In [5]:
counts = df.groupby(['instanceType', 'databaseType', 'serverType', 'requestSize', 'frequency']).count().reset_index()
counts.drop(['freeMem', 'timeWrite', 'totalMem', 'timeRead', 'timeDelete', 'totalTime', 'timestamp', 'clientTotalTime', 'memUsage'], axis=1, inplace=True)
counts.rename(columns={'id': 'count'}, inplace=True)
counts.sort_values(by=['count'], ascending=True, inplace=True)

counts['key'] = counts.apply(lambda x: '{} {} {} {}KB {}Hz'.format(x.instanceType, x.serverType, x.databaseType, x.requestSize, x.frequency), axis=1)
counts.head()

,instanceType,databaseType,serverType,requestSize,frequency,count,key
261,b1,sql,nodejs,200.0,60.0,51,b1 nodejs sql 200.0KB 60.0Hz
127,b1,nosql,nodejs,200.0,60.0,53,b1 nodejs nosql 200.0KB 60.0Hz
397,b2,nosql,nodejs,200.0,60.0,56,b2 nodejs nosql 200.0KB 60.0Hz
177,b1,sql,java,100.0,2.0,57,b1 java sql 100.0KB 2.0Hz
532,b2,sql,nodejs,200.0,60.0,61,b2 nodejs sql 200.0KB 60.0Hz


In [7]:
low_counts = counts.loc[(counts['count'] < 200)].reset_index()
low_counts.drop(['key', 'index'], axis=1, inplace=True)
low_counts = low_counts.groupby(['instanceType']).agg(['unique']).reset_index()
low_counts.columns = [col[0].strip() for col in low_counts.columns.values]

low_counts['databaseType'] = low_counts['databaseType'].apply(sorted)
low_counts['serverType'] = low_counts['serverType'].apply(sorted)
low_counts['requestSize'] = low_counts['requestSize'].apply(sorted)
low_counts['frequency'] = low_counts['frequency'].apply(sorted)
low_counts['count'] = low_counts['count'].apply(sorted)

low_counts.to_csv('data/low-count-requests.csv')
low_counts.head()

,instanceType,databaseType,serverType,requestSize,frequency,count
0,b1,"[nosql, sql]","[java, nodejs]","[1.0, 5.0, 20.0, 50.0, 100.0, 200.0, 500.0]","[1.0, 2.0, 3.0, 5.0, 10.0, 15.0, 40.0, 60.0]","[51, 53, 57, 65, 66, 71, 73, 75, 79, 83, 88, 8..."
1,b2,"[nosql, sql]","[java, nodejs]","[100.0, 200.0, 500.0]","[1.0, 10.0, 15.0, 40.0, 60.0]","[56, 61, 66, 67, 92, 101, 117, 119, 121, 187, ..."
2,b4,"[nosql, sql]","[java, nodejs]","[100.0, 200.0, 500.0]","[1.0, 40.0, 60.0]","[110, 119, 160, 161, 185]"
3,b8,"[nosql, sql]","[java, nodejs]","[100.0, 200.0, 500.0]",[1.0],[119]
